<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Conectar-área-de-trabajo" data-toc-modified-id="Conectar-área-de-trabajo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Conectar área de trabajo</a></span></li><li><span><a href="#Crear-un-script-de-entrenamiento" data-toc-modified-id="Crear-un-script-de-entrenamiento-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Crear un script de entrenamiento</a></span></li><li><span><a href="#Ejecutar-entrenamiento-en-un-script-como-experimento" data-toc-modified-id="Ejecutar-entrenamiento-en-un-script-como-experimento-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ejecutar entrenamiento en un script como experimento</a></span></li><li><span><a href="#Registrar-el-modelo-entrenado" data-toc-modified-id="Registrar-el-modelo-entrenado-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Registrar el modelo entrenado</a></span></li><li><span><a href="#Crear-a-script-parametrizado-de-training" data-toc-modified-id="Crear-a-script-parametrizado-de-training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Crear a script parametrizado de training</a></span></li><li><span><a href="#Ejecutar-script-con-argumentos" data-toc-modified-id="Ejecutar-script-con-argumentos-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Ejecutar script con argumentos</a></span></li><li><span><a href="#Registrar-nueva-versión-del-modelo" data-toc-modified-id="Registrar-nueva-versión-del-modelo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Registrar nueva versión del modelo</a></span></li></ul></div>

# Entrenar un modelo

El principal objetivo del machine learning es entrenar modelos predictivos que puedan ser usados en aplicaciones. En Azure ML, se pueden entrenar modelos con los principales frameworks scikit-learn, Tensorflow, PyTprch, SparkML, etc.

## Conectar área de trabajo

In [7]:
import azureml.core
from azureml.core import Workspace

# Cargar el área de trabajo del fichero de configuración
ws = Workspace.from_config()
print('Versión de Azure ML {} y área de trabajo {}'.format(azureml.core.VERSION, ws.name))

Versión de Azure ML 1.32.0 y área de trabajo aml-nuclio


## Crear un script de entrenamiento

Vamos a usar un script de python para entrenar un modelo de machine learning base para predecir si la calidad del vino es óptima.

In [26]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'experimentos_vino_training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/winequality.csv', os.path.join(training_folder, "winequality.csv"))

'experimentos_vino_training/winequality.csv'

In [27]:
%%writefile $training_folder/vino_training.py
# Import librerías
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Contexto de la ejecución del experimento
run = Run.get_context()

# Cargar los datos
print("Cargando los datos...")
data = pd.read_csv('winequality.csv', sep=';', decimal='.')

# Cambiar formato de la variable
data["alcohol"] = pd.to_numeric(data["alcohol"], errors='coerce')

# Eliminar nulos
data = data.dropna()

# Separar features and target
features = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']
X, y = data[features].values, data['top_quality'].values

# Separar train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Parametro de regularización
reg = 0.01

# Entrenamiento de una regresión logística
print('Entrenamiento de una Regresión Logística con ratio de regularización de', reg)
run.log('Ratio de regularizacion',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calcular el accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Guardar el modelo entrenado en la carpeta outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/vino_model.pkl')

run.complete()

Overwriting experimentos_vino_training/vino_training.py


## Ejecutar entrenamiento en un script como experimento

Ahora vamos a ejecutar el script como un experimento. El environment por defecto no incluye **scikit-learn**, entonces vamos a especificarlo para añadirlo en la configuración. El environment de conda se construye a demanda por lo tanto la primera vez que ejecute el experimento tardará algo más y luego quedará guardado para futuras ocasiones.

In [28]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails

# Crear el environment de python
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Crear el script de configuración
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='vino_training.py',
                                environment=env) 

# submit la ejecución del experimento
experiment_name = 'nuclio-vino-train'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Mostrar la ejecución del experimento como un widget en el notebook
RunDetails(run).show()

# Bloquear hasta que se complete la ejecución
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'nuclio-vino-train_1629736474_40ea0ceb',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-23T16:34:41.80082Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '486850ef-279e-4ced-b2d4-121bdcecfd06'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'vino_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'experiment_env',
   'version': 'Autosave_2021-08-23T15:00:54Z_7e64fd3a',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'dependencies': ['python=3.6.2',
      '

In [29]:
# Conseguir las métricas y ficheros
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Ratio de regularizacion 0.01
Accuracy 0.8047945205479452
AUC 0.7697446671335356


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/11357_azureml.log
outputs/vino_model.pkl


## Registrar el modelo entrenado

Observa que el output del experimento incluye el pickle del modelo entrenado **vino_model.pkl**. Ahora podemos registrar el modelo en el área de trabajo de Azure ML, haciendo posible el track de las versiones del modelo.

In [30]:
from azureml.core import Model

# Registrar el modelo
run.register_model(model_path='outputs/vino_model.pkl', model_name='vino_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# Lista de registro de modelos
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

vino_model version: 1
	 Training context : Script
	 AUC : 0.7697446671335356
	 Accuracy : 0.8047945205479452


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


AutoML2c70a24fc0 version: 1




> **Nota**: El código de arriba usa  *properties* y *tags* para añadir información adicional. La principal diferencia es que las propiedades no pueden cambiar desde de que el modelo esta registrado, mientras que los tags sí.

## Crear a script parametrizado de training

Podemos aumentar la felxibilidad del entrenamiento en el experimento añadiendo parámetros al script, permitiendo repetir el mismo experimento de training con diferentes settings. En este caso, vamos a añadir el parámetro de la regularización a la regresión logística.

El argumento es leído usando un objeto de python **argparse.ArgumentParser**.

In [31]:
%%writefile $training_folder/experimento_vino_training_params.py
# Import librerías
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Contexto de la ejecución del experimento
run = Run.get_context()

# Obtener el hiperparametro de regularización
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# Cargar los datos
print("Cargando los datos...")
data = pd.read_csv('winequality.csv', sep=';', decimal='.')

# Cambiar formato de la variable
data["alcohol"] = pd.to_numeric(data["alcohol"], errors='coerce')

# Eliminar nulos
data = data.dropna()

# Separar features and target
features = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']
X, y = data[features].values, data['top_quality'].values

# Separar train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Entrenamiento de una regresión logística
print('Entrenamiento de una Regresión Logística con ratio de regularización de', reg)
run.log('Ratio de regularizacion',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calcular el accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Guardar el modelo entrenado en la carpeta outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/vino_model.pkl')

run.complete()

Overwriting experimentos_vino_training/experimento_vino_training_params.py


## Ejecutar script con argumentos

In [32]:
# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='experimento_vino_training_params.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=env) 

# submit the experiment
experiment_name = 'nuclio-vino-train'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'nuclio-vino-train_1629736589_e756789f',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-23T16:37:36.066313Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '486850ef-279e-4ced-b2d4-121bdcecfd06'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'experimento_vino_training_params.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg_rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'experiment_env',
   'version': 'Autosave_2021-08-23T15:00:54Z_7e64fd3a',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'

In [33]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
    print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Ratio de regularizacion 0.1
Accuracy 0.8068493150684931
AUC 0.7698252576116318


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/11511_azureml.log
outputs/vino_model.pkl


## Registrar nueva versión del modelo

In [34]:
from azureml.core import Model

# Registrar modelo
run.register_model(model_path='outputs/vino_model.pkl', model_name='vino_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# Lista de modelos registrados
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

vino_model version: 2
	 Training context : Parameterized script
	 AUC : 0.7698252576116318
	 Accuracy : 0.8068493150684931


vino_model version: 1
	 Training context : Script
	 AUC : 0.7697446671335356
	 Accuracy : 0.8047945205479452


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


AutoML2c70a24fc0 version: 1


